In [ ]:
import numpy as np 
import pandas as pd 
from numpy import unique
from numpy import where
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.cluster import Birch
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
%matplotlib inline
import matplotlib.pyplot as plt


In [ ]:
train = pd.read_csv('../input/tabular-playground-series-apr-2021/train.csv')


<h1> Simple Features engineering <h1>

In [ ]:
train['Embarked'] = train['Embarked'].fillna('Z')
train['Cabin'] = train['Cabin'].fillna('_')
train.Ticket = train.Ticket.map(lambda x:str(x).split()[0] if len(str(x).split()) > 1 else 'X')
train['CabinType'] = train['Cabin'].apply(lambda x:x[0])
train['Age'].fillna(round(train['Age'].mean()), inplace=True,)
train['Age'] = train['Age'].apply(round)
train['Age'] = train['Age'].astype(np.int8)
train['Fare'].fillna(round(train['Fare'].mean()), inplace=True,)

In [ ]:
def label_encoder(c):
    lc = LabelEncoder()
    return lc.fit_transform(c)

In [ ]:
label_cols = ['Ticket','CabinType','Embarked']
numerical_cols_train = ['Pclass','Fare', 'Age','Sex','Survived']

In [ ]:
numerical_train = train[numerical_cols_train]
label_encoded_train = train[label_cols].apply(label_encoder)

In [ ]:
train = pd.concat([numerical_train, label_encoded_train], axis=1)

In [ ]:
dico={'male':0,'female':1}

In [ ]:
train['Sex']=train['Sex'].map(dico)

In [ ]:
lc = LabelEncoder()
MinMax=MinMaxScaler()
for col in train.columns:
    train[col] = pd.DataFrame(MinMax.fit_transform(lc.fit_transform(train[col]).reshape(-1,1)),columns=[col])


In [ ]:
X=train[['Pclass','Fare', 'Age','Sex','CabinType','Embarked','Survived']]

In [ ]:
X.head(2)

<h1> Clustering <h1>

https://openclassrooms.com/en/courses/5869986-perform-an-exploratory-data-analysis/6177851-carry-out-a-k-means-clustering

In [ ]:
# Create a k-means clustering model
kmeans = KMeans(init='random', n_clusters=3, n_init=10)

# Fit the data to the model
kmeans.fit(X)

# Determine which clusters each data point belongs to:
clusters =  kmeans.predict(X)

In [ ]:
# Add cluster number to the original data
X_clustered = pd.DataFrame(X, columns=X.columns, index=X.index)
X_clustered['cluster'] = clusters

X_clustered.head()

In [ ]:
# Display parallel coordinates plots, one for each cluster
plt.figure(figsize=(14,8),dpi=80)
pd.plotting.parallel_coordinates(X_clustered,'cluster', color=('red', 'blue', 'green'))

We can see:
- the cluster 1 (red) is mainly female, old, in class 1
- the cluster 2 (blue) is mainly male, young, in Class 3, who didn't survive
- the cluster 3 (green) is mainly female who survived

At the end of the script, the last picture is clearer

<h1> Determine the optimal number of clusters with the elbow curve : <h1>

In [ ]:
# Run a number of tests, for 1, 2, ... num_clusters
num_clusters = 20
kmeans_tests = [KMeans(n_clusters=i, init='random', n_init=10) for i in range(1, num_clusters)]
score = [kmeans_tests[i].fit(X).score(X) for i in range(len(kmeans_tests))]

# Plot the curve
plt.figure(figsize=(12,6),dpi=80)
plt.plot(range(1, num_clusters),score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show()

In [ ]:
# the elbow shows that we may choose around 3 or 4 clusters

In [ ]:
# Create a data frame containing our centroids
centroids = pd.DataFrame(kmeans.cluster_centers_, columns=X_clustered.columns)
centroids['cluster'] = centroids.index


In [ ]:
plt.figure(figsize=(14,8),dpi=80)
pd.plotting.parallel_coordinates(centroids,'cluster',color=('red', 'blue', 'green'))